In [1]:
# authorize access to the drive
# This allows me to access files (Data) in my google drive
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
# This downloads Hugging Face Transformers, tools we need to fine-tune GPT-2
!pip3 install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-pbj2go5y
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-pbj2go5y
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 20.4MB/s 
     |████████████████████████████████| 890kB 52.9MB/s 
     |████████████████████████████████| 2.9MB 49.9MB/s 
  Created wheel for transformers: filename=transformers-3.5.0-cp36-none-any.whl size=1311439 sha256=0cf8ecb5216634e66b412f9dbfccb8b34532a8097fc0030aacee7bec6f79763b
  Stored in directory: /tmp/pip-ephem-wheel-cache-yy8sa5jq/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=7f5d1df2143c0c04645c91644bd588098b226877371fee239627f150a084c42d
  Stored in directory: /root/.cach

In [3]:
# Allows me to go to the correct directory in my google drie
%cd drive/My\ Drive/School\ Work/Fourth\ Year/Capstone\ Everything

/content/drive/My Drive/School Work/Fourth Year/Capstone Everything


In [4]:
# the fine-tuning code by Hugging Face that allows us to fine-tune
# this file is already archieved after the most recent hugging face modelling se we need to replace the url
# !wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/contrib/legacy/run_language_modeling.py

--2020-11-14 12:32:37--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-11-14 12:32:37 ERROR 404: Not Found.



In [5]:
% ls

'Brian'\''s Peer Feedback Template (10.28.21).gdoc'
'Capstone Coversheet.gdoc'
'Capstone Proposal (Junior Year).gdoc'
 class/
'Dev Journal.gdoc'
 Models/
 None/
 output_description/
'Paper Drafts'/
'Punch Card.gsheet'
 run_language_modeling.py
 run_language_modeling.py.1
 run_language_modeling.py.2
 run_language_modeling.py.3
 runs/


In [6]:
# get dataset
datapath = "Models/Data"
train_path = datapath + "/train.txt"
valid_path = datapath + "/valid.txt"
output = "./output"

In [ ]:
# %%bash 
# export DIRECTORY=Models/Data/Description_Data
# ls $DIRECTORY

cached_lm_GPT2Tokenizer_1024_description_train.txt
cached_lm_GPT2Tokenizer_1024_description_train.txt.lock
cached_lm_GPT2Tokenizer_1024_description_valid.txt
cached_lm_GPT2Tokenizer_1024_description_valid.txt.lock
description_test.txt
description_train.txt
description_valid.txt


In [ ]:
# Main fine-tuning code using run_language_modeling.py
%%bash
export TRAIN_FILE=Models/Data/train.txt
export TEST_FILE=Models/Data/valid.txt
export MODEL_NAME=gpt2
export OUTPUT_DIR=Models/Claims_model

# python3 run_language_modeling.py \
#     --output_dir=$OUTPUT_DIR \
#     --model_type=$MODEL_NAME \
#     --model_name_or_path=$MODEL_NAME \
#     --do_train \
#     --train_data_file=$TRAIN_FILE \
#     --do_eval \
#     --eval_data_file=$TEST_FILE \
#     --per_gpu_train_batch_size=1 \
#     --save_steps=-1 \
#     --num_train_epochs=2

python run_language_modeling.py \
    --output_dir=$OUTPUT_DIR \
    --model_type=gpt2 \
    --model_name_or_path=$MODEL_NAME \
    --do_train \
    --train_data_file=$TRAIN_FILE \
    --do_eval \
    --eval_data_file=$TEST_FILE \
    --per_gpu_train_batch_size=1 \
    --save_steps=-1 \
    --num_train_epochs=2

In [ ]:
# this allow us to generate sample patent claims 

from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import numpy as np

OUTPUT_DIR = "./output_description"
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
# we use the GPT-2 model to load our fine-tuned model and get the tokenization
tokenizer = GPT2Tokenizer.from_pretrained(OUTPUT_DIR)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)
model = model.to(device)
                                        
def generate(input_str, length=250, n=5):
  '''
  This is the main generation code using our model
  '''
  cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)
  model.eval()
  with torch.no_grad():
    for i in range(length):
      outputs = model(cur_ids[:, -1024:], labels=cur_ids[:, -1024:])
      loss, logits = outputs[:2]
      softmax_logits = torch.softmax(logits[0,-1], dim=0)
      next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
      cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
    output_list = list(cur_ids.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
    return output_text

def choose_from_top(probs, n=5):
    '''
    This chooses the top n choices and probabilistically select one choice
    '''
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)
# print the generated text
generated_text = generate("This aparatus \n", n=5)
print(generated_text)

This aparatus 
<BOS> The present invention relates generally to organic electroluminescent devices, and more particularly to      an organic electroluminescent device comprising: a base      electrode; a cathode; and an an electron transport region between the      base electrode and the electron transport region. The electron transport      region includes a first region separated from the base electrode by a junction,      and a second region separated from the cathode by a junction. The      junction includes a first material in contact with the first region and a      second material in contact with the second region. The junction includes a      third material in contact with the third material in contact with the      junction. <EOS>
<BOS> A method of manufacturing a light emitting device having a resin package      which provides an optical reflectivity equal to or more than 70% at a      wavelength between 350 nm and 800 nm after thermal curing, and in which a      resin part a

In [ ]:
% ls output

config.json          merges.txt               tokenizer_config.json
eval_results_lm.txt  pytorch_model.bin        training_args.bin
log_history.json     special_tokens_map.json  vocab.json
